In [2]:
import numpy as np # scientific computing
import pandas as pd # data loading and processing
import os # os operations
import matplotlib.pyplot as plt # for generating figures
import math
import matplotlib.dates as mdates
import seaborn as sns # for generating visualizations, better support with pandas than matplotlib
from scipy import stats

In [3]:
# get names of nrf2 target genes
nrf2_names = pd.read_csv("./data/nrf2 target genes.csv", index_col=None, header= None).T
nrf2_names = nrf2_names.loc[0].to_list()

gene_set = pd.read_csv("./data/oxstress genes.csv", index_col=None, header= 0)
gene_set = gene_set['NRF2 v2'].dropna().tolist()

targets = list(set(["G6PD", "RRM2B"] +  gene_set))

In [36]:
df = pd.read_csv("./data/HCCDB/HCCDB1_mRNA_level3.txt", index_col=1, sep = "\t").drop(["Entrez_ID"], axis=1) # gene x patient
df

,HCCDB-1.S1,HCCDB-1.S2,HCCDB-1.S3,HCCDB-1.S4,HCCDB-1.S5,HCCDB-1.S6,HCCDB-1.S7,HCCDB-1.S8,HCCDB-1.S9,HCCDB-1.S10,...,HCCDB-1.S188,HCCDB-1.S189,HCCDB-1.S190,HCCDB-1.S191,HCCDB-1.S192,HCCDB-1.S193,HCCDB-1.S194,HCCDB-1.S195,HCCDB-1.S196,HCCDB-1.S197
Symbol,,,,,,,,,,,,,,,,,,,,,
PDXDC1,8.18040,7.98420,8.81030,7.62550,6.3780,9.14260,8.5225,6.10330,6.27840,8.85930,...,6.49030,8.24680,8.07420,6.39690,6.58460,8.32410,7.83720,7.85730,8.14640,6.69080
DGKK,3.12565,3.05915,2.79290,3.04215,3.0406,2.91910,3.1426,2.70120,2.98070,3.04185,...,3.30210,2.76015,3.18305,3.65615,2.96310,2.69595,2.98670,2.86125,3.03875,3.17505
ZNF417,5.72130,5.91835,5.68760,5.97510,6.3881,5.96585,5.5097,6.03590,5.50950,5.44995,...,5.72740,6.15445,5.53945,6.20515,5.32815,6.12640,5.73380,5.43425,5.88720,5.43570
PTCD3,8.82480,8.19005,8.37045,8.49940,8.0901,8.12535,8.3818,9.12295,8.16705,9.23080,...,7.99915,7.87900,8.20370,8.30205,7.68480,8.00135,8.22035,8.22115,7.58460,7.94505
SPTBN1,9.84270,9.96940,9.72280,9.68470,10.6949,9.98890,10.2911,8.94710,10.54550,9.98440,...,9.74130,8.98530,9.50720,9.78250,9.72510,10.65740,9.52990,9.78650,10.04230,10.01720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SNORA70,8.57170,9.40990,10.06560,9.82630,9.5958,8.74580,8.8302,8.85150,9.36520,8.43820,...,8.34170,8.97410,8.75650,8.97380,8.34690,9.25120,8.62580,8.59100,8.87730,8.70630
SNORA71A,3.60220,3.36180,3.42210,3.48770,3.5584,3.45770,3.4389,3.74450,3.51480,3.76520,...,3.79900,3.42050,3.59010,3.55950,3.51660,3.66400,3.69880,3.90070,3.54830,3.49850
SLC25A17,9.90560,9.64950,10.04810,10.16470,10.0405,9.24670,9.9690,9.27330,10.11860,9.88080,...,10.17360,9.74250,10.03290,9.19840,9.68960,10.06050,10.27710,10.06820,9.75270,10.07220


In [37]:
sum(df.index.duplicated())

3626

In [38]:
# df = df.loc[~(df.index == "---"),:].copy()

In [39]:
df = df.loc[~df.index.duplicated(),:].copy() 

In [40]:
sum(df.index.duplicated())

0

In [35]:
df.loc[df.index.duplicated(),:]

,HCCDB-1.S1,HCCDB-1.S2,HCCDB-1.S3,HCCDB-1.S4,HCCDB-1.S5,HCCDB-1.S6,HCCDB-1.S7,HCCDB-1.S8,HCCDB-1.S9,HCCDB-1.S10,...,HCCDB-1.S188,HCCDB-1.S189,HCCDB-1.S190,HCCDB-1.S191,HCCDB-1.S192,HCCDB-1.S193,HCCDB-1.S194,HCCDB-1.S195,HCCDB-1.S196,HCCDB-1.S197
Symbol,,,,,,,,,,,,,,,,,,,,,
C21orf122,5.10860,5.1149,4.86650,5.1640,4.99830,4.96560,5.3693,5.32230,5.09370,5.0211,...,5.0645,4.95430,5.33060,5.06920,5.0855,4.65850,5.3180,5.2534,4.9610,5.50370
DNAH14,4.93205,5.3401,5.68005,5.1267,5.45925,4.77205,5.0982,5.16595,4.80635,5.0313,...,5.4355,4.83565,5.15635,6.21985,5.2628,5.68585,5.0692,5.4174,4.6268,5.34325


In [43]:
df2 = pd.read_csv("./data/HCCDB/HCCDB3_mRNA_level3.txt", index_col=1, sep = "\t").drop(["Entrez_ID"], axis=1) # gene x patient
df2

,HCCDB-3.S1,HCCDB-3.S2,HCCDB-3.S3,HCCDB-3.S4,HCCDB-3.S5,HCCDB-3.S6,HCCDB-3.S7,HCCDB-3.S8,HCCDB-3.S9,HCCDB-3.S10,...,HCCDB-3.S548,HCCDB-3.S549,HCCDB-3.S550,HCCDB-3.S551,HCCDB-3.S552,HCCDB-3.S553,HCCDB-3.S554,HCCDB-3.S555,HCCDB-3.S556,HCCDB-3.S557
Symbol,,,,,,,,,,,,,,,,,,,,,
CAB39L,0.1935,0.1305,0.064,0.1715,0.143,0.119,0.149,0.166,0.1145,0.091,...,0.157,0.171,0.0545,0.1415,0.0795,0.177,0.103,0.1095,0.092,0.139
MPV17L,2.1770,1.5580,3.290,2.5260,3.160,7.346,2.397,3.417,3.0290,5.206,...,0.548,0.586,4.7300,0.9320,1.9710,0.914,0.840,1.2410,1.002,0.313
ALDH8A1,16.2890,15.3080,14.984,15.9210,13.073,11.790,14.334,12.592,14.9920,16.458,...,3.333,9.248,2.5320,15.8740,4.0170,8.639,7.383,11.5580,2.693,4.515
VSIG1,0.0090,0.0090,0.009,0.0130,0.008,0.016,0.010,0.009,0.0160,0.009,...,0.012,0.018,0.0150,0.0150,0.0100,0.016,0.008,0.0210,0.008,0.012
PLD3,0.6240,1.0450,0.948,0.9650,0.943,1.050,0.803,0.689,0.9110,1.026,...,7.979,1.113,1.5320,0.6510,1.5160,0.993,1.298,1.2930,0.617,0.780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
---,0.0370,0.0470,0.056,0.0520,0.031,0.053,0.046,0.046,0.0550,0.048,...,0.034,0.034,0.0420,0.0370,0.0450,0.052,0.045,0.0530,0.099,0.043
GIMAP5,2.8620,3.6650,1.553,3.1080,4.086,3.022,6.112,5.097,6.0390,5.313,...,3.961,1.679,4.9430,5.6160,1.0330,1.838,3.530,3.9420,1.798,3.777
---,0.0300,0.0320,0.054,0.0770,0.036,0.059,0.024,0.029,0.0260,0.037,...,0.017,0.013,0.0230,0.1470,0.0490,0.025,0.019,0.0270,0.049,0.021


In [44]:
print(sum(df2.index.duplicated()))
df2 = df2.loc[~df2.index.duplicated(),:].copy() 
print(sum(df2.index.duplicated()))

2081
0


In [45]:
df = pd.concat([df, df2], axis = 1) # patients x genes

array([ True,  True,  True, ...,  True,  True, False])

In [24]:
df1 = pd.read_csv("./data/HCCDB/HCCDB15.sample.txt", index_col=0,  sep = "\t").T # gene x patient
df1.index

Index(['HCCDB-15.S1', 'HCCDB-15.S2', 'HCCDB-15.S3', 'HCCDB-15.S4',
       'HCCDB-15.S5', 'HCCDB-15.S6', 'HCCDB-15.S7', 'HCCDB-15.S8',
       'HCCDB-15.S9', 'HCCDB-15.S10',
       ...
       'HCCDB-15.S396', 'HCCDB-15.S397', 'HCCDB-15.S398', 'HCCDB-15.S399',
       'HCCDB-15.S400', 'HCCDB-15.S401', 'HCCDB-15.S402', 'HCCDB-15.S403',
       'HCCDB-15.S404', 'HCCDB-15.S405'],
      dtype='object', length=405)

In [25]:
# subset to get relevant genes
df_filtered = df.loc[targets]
# print(df_filtered.shape)
df_filtered




KeyError: "['GSTM2P1', 'GSTM2', 'PTGR1'] not in index"

In [ ]:
# check for nan values
na_filter = df_filtered.isnull().any()
na_ls = na_filter[na_filter == True].index.to_list()
print(na_ls)

# impute nan values
mean_ls = df_filtered.mean()
for na_col in na_ls:
    df_filtered[na_col] = df_filtered[na_col].fillna(mean_ls[na_col])
    print(df_filtered.isnull().values.any())
df_filtered = df_filtered.T # patients x genes

# scale numerical data
df_filtered = np.log10(df_filtered+1)

# for each sequenced gene were rescaled to set the median equal to 1
data=((df_filtered-df_filtered.median())/df_filtered.std())+1

# take the mean across nrf2 target genes to get nrf2 activity 
nrf2 = data[gene_set]
data.drop(gene_set, inplace = True, axis = 1)
nrf2["NFE2L2"] = nrf2.mean(axis = 1)
data = pd.concat([data, nrf2], axis = 1) # patients x genes 

#find line of best fit
y, x = data["NFE2L2"].to_numpy(), data["RRM2B"].to_numpy()
a, b = np.polyfit(x, y, 1)

iqr = data["RRM2B"].T.describe()

# bin the patients into quartiles based on G6PD expression
data["RRM2B levels"] = pd.cut(data["RRM2B"],
                bins=[ iqr["min"], iqr["25%"], iqr["75%"], iqr["max"]],
                labels=["Bottom 25%", "-", "Top 25%"])

# get r sq val
r = np.corrcoef(x, y)[0, 1]


#find p-value
n = data.shape[0]
t = (r-math.sqrt(n-2))/math.sqrt(1-(r**2))
p = stats.t.sf(abs(t), df=n)*2

# plot the data

# scatter plot for RRM2B against NRF2 activity
sns.set_style("whitegrid")
sns.set()
sns.scatterplot(data=data, x="RRM2B", y="NFE2L2", color = "green")
plt.plot(x, a*x+b, color="black")
# axs[cancer.index(c)].set_ylabel("NRF2 activity",fontsize = 18)
# axs[cancer.index(c)].set_xlabel("RRM2B expression" + " \n (r = " + str(round(r, 4)) + "," + " p = " + str(round(p, 4)) +")",rotation=10,fontsize = 15)
# axs[cancer.index(c)].set_title(cancer[cancer.index(c)], fontsize = 20)
# axs[cancer.index(c)].tick_params(axis='both', which='major', labelsize=15)
plt.show()
